In [2]:
from olsjan10 import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
X = get_fundamental_data('EPS_TTM')
rowindex = X.index
colindex = X.columns
X.dropna(axis=0,inplace=True)

In [18]:
# 计算协方差矩阵
eigen_vals, eigen_vectors = np.linalg.eig(np.cov(X,rowvar=False))

In [23]:
# 提取4,5个隐含宏观因子
eigen_vals.cumsum() / eigen_vals.sum()

array([0.42904724, 0.58949993, 0.75919926, 0.84981602, 0.91199379,
       0.9482026 , 0.97142685, 0.98096498, 0.98843618, 0.99376678,
       0.99616512, 0.99733796, 0.99800106, 0.99862374, 0.99908947,
       0.99941641, 0.99961916, 0.99976565, 0.99987157, 0.99991041,
       0.99993854, 0.99995601, 0.99996906, 0.99997992, 0.99998577,
       0.99999046, 0.99999475, 0.99999501, 0.99999731, 0.99999879,
       1.        ])

In [27]:
Factors = X @ eigen_vectors

## 市场隐含宏观因子

这种做法的频率更高，能够进行实时反馈，而且对资产的解释力度通常更高。(和之前对比的话，4个隐含因子和70个宏观经济变量因子的解释力度相差无几)

根据主成分的权重, 可以对因子进行命名(多为增长，通胀，利率)，但存在的问题是，我们的资产是行业指数，通过行业指数的权重能够进行命名吗。

In [51]:
def cal_implicit_factors(asset_name, n=4, method=None):
    """
    Parameters
    -----------
    asset_name: str
        EPS_TTM, PE_TTM
    n: int
    """
    X = get_fundamental_data(f'{asset_name}')
    rowindex = X.index
    colindex = X.columns
    X.dropna(axis=0,inplace=True)
    eigen_vals, eigen_vectors = np.linalg.eig(np.cov(X,rowvar=False))
    Variance_interpretation_ratio = eigen_vals.cumsum() / eigen_vals.sum()
    Factors = X @ eigen_vectors
    Factors.columns = [f"pca{i+1}" for i in range(len(X.columns))]
    return Factors.iloc[:,:n], Variance_interpretation_ratio

In [52]:
factors, _ = cal_implicit_factors('EPS_TTM')

In [53]:
factors

,pca1,pca2,pca3,pca4
2015-03-31,-0.088453,-0.137454,-0.087782,-0.174349
2015-04-30,-0.079116,-0.121234,-0.083903,-0.186819
2015-05-29,-0.082552,-0.250133,-0.117578,-0.200076
2015-06-30,-0.122651,-0.850750,-0.097451,0.269398
2015-07-31,-0.113598,-0.842315,-0.092122,0.298394
...,...,...,...,...
2022-09-30,-4.537572,1.330841,-1.102571,3.396449
2022-10-31,-4.538129,1.330714,-1.095954,3.394226
2022-11-30,-4.521406,1.332091,-1.092563,3.393975
2022-12-30,-3.596759,-0.365759,-0.313048,0.071834


## 法二